## Please pip install all files 
make sure all necessary files are installed 

In [26]:
# pip install lyricsgenius
# pip install nltk
# pip install textBlob
# there may be some left over imports from previous revisions
import nltk
import pandas as pd
import csv
import re
import toolz
import lyricsgenius
import altair as alt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
nltk.download('wordnet')
import numpy as np

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Please give time for code to run may take 3-5 minutes
New data set will be created so having Polarity 1.csv is the only file neccesary, If for some reason it does not pop up just install the polarity Dataset-Sheet.csv

In [25]:
lemmatizer = WordNetLemmatizer()

# Genius api key 
api_key = None  

with open('API key.txt', 'r') as api_file:
    api_key = api_file.read().strip() 
genius = lyricsgenius.Genius(api_key)
# genius = lyricsgenius.Genius('K1dNXCltkmB-ndrRoyD5DV6_MmevzqXmzJC_cVyLxl0f93L2DDDFz3zki3h8gLG4') 
# if file doesn't work just activate code above or use your own api key

# This function will fetch song lyrics, lemmatize them, and run a sentiment analysis
def process_title(title):
    if not title.strip(): 
        return None
    
    song = genius.search_song(title, 'Bones')
    if song:
        lyrics = song.lyrics
        # Lemmatize the lyrics
        lemmatized_lyrics = ' '.join([lemmatizer.lemmatize(word) for word in re.sub(r"\n{1,2}", ". ", re.sub(r"\?\n{1,2}", "? ", lyrics)).split()])
        sentiment_polarity = TextBlob(lemmatized_lyrics).sentiment.polarity
        return sentiment_polarity
    else:
        return None
# There are blank spaces in the file that have to be taken account for 
    
# Open original csv file
with open('Polarity 1.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader)
    value_column_index = header.index("value")

    # new columns are created for the name and value, the original value column held the song names which are transferred to the new name column
    # Original value column is now sentiment polarity values
    
    updated_rows = [header[:value_column_index] + ["name", "value"] + header[value_column_index + 1:]]  
    for row in csvreader:
        sentiment_polarity_values = list(row)  
        name_value = sentiment_polarity_values[value_column_index]  
        # Activate sentiment polarity analysis 
        sentiment_polarity = process_title(name_value)
        # enter in new values to columns
        updated_rows.append(sentiment_polarity_values[:value_column_index] + [name_value, sentiment_polarity] + sentiment_polarity_values[value_column_index + 1:])

# this will create a CSV file so having this file prior to running code is not necessary 
output_file = 'Polarity Dataset - Sheet1.csv'
with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(updated_rows)
# write in new data to new csv file

# print to let user know that program has finished running
print(f'A new "name" column with original data and the "value" column with sentiment polarity have been added to {output_file}')


Searching for ""Zig Zags And Louie Tags"" by Bones...
Done.
Searching for "Weekend" by Bones...
Done.
Searching for "Mighty Joe Young" by Bones...
Done.
Searching for "Excuse Me" by Bones...
Done.
Searching for "Work" by Bones...
Done.
Searching for "MasterBedroom" by Bones...
Done.
Searching for "MarbleFloors" by Bones...
Done.
Searching for "BluntsInTheGraveyard" by Bones...
Done.
Searching for "BabyBlueVeleour" by Bones...
Done.
Searching for "SilkSheets" by Bones...
Done.
Searching for "TheLivingForest" by Bones...
Done.
Searching for "Phantom" by Bones...
Done.
Searching for "Gold Chevrolet" by Bones...
Done.
Searching for "White Noise" by Bones...
Done.
Searching for "Windows 95" by Bones...
Done.
Searching for "Dead Girl" by Bones...
Done.
Searching for "Pixelated Tears" by Bones...
Done.
Searching for "Penthouse" by Bones...
Done.
Searching for "Dracula" by Bones...
Done.
Searching for "Floppy Disc" by Bones...
Done.
Searching for "Marble  " by Bones...
Done.
Searching for "Jon

## Heat map Below Sentiment Polarity Skull format
Tool tip is included but only works in html format so please use html link if you want to fully interact with the visualization

Output may look slightly different depending on your coding compiler

sentiment polarity goes -1,1(-1 most negative lyrics, 1 positive lyrics

The bottom of this ipynb file has more color schemes 

In [40]:
data = pd.read_csv('Polarity Dataset - Sheet1.csv')
# read in new csv file 
# Heat map skull pattern creation 
heatmap = alt.Chart(data).mark_rect().encode(
    x=alt.X('number:O', title=None,axis=None),  # remove title and axis for cleaner look
    y=alt.Y('album :N', title='Album', sort=None), # set sorting to none, very necessary for skull pattern, for some reason without it, it sorts randomly
    color=alt.Color('value:Q', scale=alt.Scale(domain=[-.5, .5], scheme='inferno')),# set color and create a value scale for a better look
    tooltip=['value:Q', 'album :N','name:N'] # interactive component
).properties(
    width=400,
    height=400
)
heatmap
 

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

## Average mean sentiment per album over years
Ironically some of the darker albums are slighlty positive like teenwitch

In [39]:
# create seperate visuals for average scores over the years
barchart = alt.Chart(data).mark_bar().encode(
    x=alt.X('mean(value):Q', title='Mean Sentiment Polarity'),
    y=alt.Y('album :N', title='Album', sort=None),
    tooltip=['mean(value):Q', 'album :N']
).properties(
    width=400,
    height=400
)
barchart

alt.Chart(...)

## Other color schemes


In [43]:
data = pd.read_csv('Polarity Dataset - Sheet1.csv')
heatmap = alt.Chart(data).mark_rect().encode(
    x=alt.X('number:O', title=None,axis=None),
    y=alt.Y('album :N', title='Album',sort=None),
    color=alt.Color('value:Q', scale=alt.Scale(domain=[-.5, .5], scheme='turbo')),
    tooltip=['value:Q', 'album :N','name:N']
).properties(
    width=400,
    height=400
)
heatmap 

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [42]:
data = pd.read_csv('Polarity Dataset - Sheet1.csv')
heatmap = alt.Chart(data).mark_rect().encode(
    x=alt.X('number:O', title=None, axis=None),
    y=alt.Y('album :N', title='Album',sort=None),  
    color=alt.Color('value:Q', scale=alt.Scale(domain=[-.5, .5], scheme='darkred')),
    tooltip=['value:Q', 'album :N','name:N']
).properties(
    width=400,
    height=400
)
heatmap 

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [45]:
data = pd.read_csv('Polarity Dataset - Sheet1.csv')

heatmap = alt.Chart(data).mark_rect().encode(
    x=alt.X('number:O', title=None, axis=None),
    y=alt.Y('album :N', title='Album',sort=None),  
    color=alt.Color('value:Q', scale=alt.Scale(domain=[-.5, .5], scheme='redgrey')),
    tooltip=['value:Q', 'album :N','name:N']
).properties(
    width=400,
    height=400
)
heatmap 

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [46]:
data = pd.read_csv('Polarity Dataset - Sheet1.csv')
heatmap = alt.Chart(data).mark_rect().encode(
    x=alt.X('number:O', title=None, axis=None),
    y=alt.Y('album :N', title='Album',sort=None),  
    color=alt.Color('value:Q', scale=alt.Scale(domain=[-.5, .5], scheme='spectral')),
    tooltip=['value:Q', 'album :N','name:N']
).properties(
    width=400,
    height=400
)
heatmap 

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)